In [30]:
import re
import pandas as pd

In [2]:
cpi = pd.read_csv('../data/03_외부데이터/CPI(2019년~2020년).csv', encoding = 'CP949')

# Category work

In [52]:
raw_cat = list(cpi.지출목적별)

In [58]:
usable_cats = raw_cat[:292] # 06.보건부터 버림

In [59]:
big_cats = list(filter(lambda x: x[0] in map(str, range(0, 10)), map(str, usable_cats)))

In [60]:
small_cats = list(filter(lambda x: re.findall(r'\d\d\.', x), big_cats))

In [61]:
small_idx = list(map(usable_cats.index, small_cats))
small_idx

[2, 137, 149, 157, 161, 194, 201, 205, 209, 217, 229, 239, 243, 262, 272, 277]

In [84]:
hier = dict()

In [88]:
for i in range(len(small_idx) - 2 + 1):
    start, end = small_idx[i:i+2]
    hier[usable_cats[start][5:]] = list(map(lambda x: x.replace(' ', ''), usable_cats[start+1:end]))

In [87]:
hier

{'식료품': ['빵및곡물',
  '쌀',
  '현미',
  '찹쌀',
  '보리쌀',
  '콩',
  '땅콩',
  '혼식곡',
  '밀가루',
  '국수',
  '라면',
  '당면',
  '두부',
  '시리얼',
  '부침가루',
  '케이크',
  '빵',
  '떡',
  '파스타면',
  '육류',
  '국산쇠고기',
  '수입쇠고기',
  '돼지고기',
  '닭고기',
  '소시지',
  '햄및베이컨',
  '어류및수산',
  '갈치',
  '명태',
  '조기',
  '고등어',
  '오징어',
  '게',
  '굴',
  '조개',
  '전복',
  '마른멸치',
  '마른오징어',
  '낙지',
  '오징어채',
  '북어채',
  '어묵',
  '맛살',
  '생선통조림',
  '젓갈',
  '우유·치즈및계란',
  '우유',
  '분유',
  '치즈',
  '발효유',
  '달걀',
  '식용유지',
  '참기름',
  '식용유',
  '과일',
  '사과',
  '배',
  '복숭아',
  '포도',
  '밤',
  '감',
  '귤',
  '오렌지',
  '참외',
  '수박',
  '딸기',
  '바나나',
  '키위',
  '블루베리',
  '아몬드',
  '과일가공품',
  '채소및해조',
  '배추',
  '상추',
  '시금치',
  '양배추',
  '미나리',
  '깻잎',
  '부추',
  '무',
  '열무',
  '당근',
  '감자',
  '고구마',
  '도라지',
  '콩나물',
  '버섯',
  '오이',
  '풋고추',
  '호박',
  '가지',
  '토마토',
  '파',
  '양파',
  '마늘',
  '브로콜리',
  '고사리',
  '파프리카',
  '단무지',
  '김',
  '맛김',
  '미역',
  '과자·빙과류및당류',
  '초콜릿',
  '사탕',
  '껌',
  '아이스크림',
  '비스킷',
  '스낵과자',
  '파이',
  '설탕',
  '잼',
  '꿀',
  '물엿',
  '기타식

In [92]:
train = pd.read_excel(r'../data/01_제공데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터.xlsx',
                   header=1,
                   names=['방송일시', '노출(분)', '마더코드', '상품코드',
                          '상품명', '상품군', '판매단가', '취급액'],
                   thousands=',',
                   )

In [93]:
prod_names = (train[train.상품군 != '무형'].상품명 + ' ' + train[train.상품군 != '무형'].상품군).values# len(prod_names) > 1692 (무형 빼고)

In [98]:
from konlpy.tag import Komoran
komoran = Komoran(userdic='../data/04_추가데이터/user_dict_0913.txt')
print(komoran.nouns("KOMORAN은 한국어 형태소 분석기입니다."))

naive_tokens = list(map(komoran.pos, prod_names))

['한국어', '형태소', '분석기']


In [99]:
def morph_filter(lst):
    return list(map(lambda x: x[0], filter(lambda x: (x[1] in usable_morphs), lst)))

usable_morphs = ['NNG', 'NNP', 'NP', 'NR'] #+ ['SL', 'SN']
naive_tokens = list(map(morph_filter, naive_tokens))

In [100]:
exclude_words = ['종', '인용']
def exclude(lst):
    return list(filter(lambda x: x not in exclude_words, lst))
processed_tokens = list(map(exclude, naive_tokens))

In [101]:
processed_tokens

[['테이트', '남성', '셀린니트', '의류'],
 ['테이트', '여성', '셀린니트', '의류'],
 ['테이트', '남성', '셀린니트', '의류'],
 ['테이트', '여성', '셀린니트', '의류'],
 ['테이트', '남성', '셀린니트', '의류'],
 ['테이트', '여성', '셀린니트', '의류'],
 ['오모떼', '레이스', '파운데이션', '브라', '속옷'],
 ['오모떼', '레이스', '파운데이션', '브라', '속옷'],
 ['오모떼', '레이스', '파운데이션', '브라', '속옷'],
 ['CERINI', 'PAT', '남성', '소프트', '기모', '릴렉스', '팬츠', '의류'],
 ['CERINI', 'PAT', '남성', '소프트', '기모', '릴렉스', '팬츠', '의류'],
 ['CERINI', 'PAT', '남성', '소프트', '기모', '릴렉스', '팬츠', '의류'],
 ['보코', '리버시블', '무스탕', '의류'],
 ['보코', '리버시블', '무스탕', '의류'],
 ['보코', '리버시블', '무스탕', '의류'],
 ['CERINI', 'PAT', '남성', '풀', '패키지', '기모', '니트', '의류'],
 ['CERINI', 'PAT', '남성', '풀', '패키지', '기모', '니트', '의류'],
 ['CERINI', 'PAT', '남성', '풀', '패키지', '기모', '니트', '의류'],
 ['크로커다일', '편안', '코튼', '브라팬티', '세트', '속옷'],
 ['크로커다일', '편안', '코튼', '브라팬티', '세트', '속옷'],
 ['에코라믹', '통주', '무', '스톤', '냄비', '세트', '주방'],
 ['에코라믹', '통주', '무', '스톤', '냄비', '세트', '주방'],
 ['에코라믹', '통주', '무', '스톤', '냄비', '세트', '주방'],
 ['오모떼', '360', '텐션', '업', '레이스', '패키지', '시즌', '

In [102]:
train.상품군.unique()

array(['의류', '속옷', '주방', '농수축', '이미용', '가전', '생활용품', '건강기능', '잡화', '무형',
       '가구', '침구'], dtype=object)

In [103]:
hier.keys()

dict_keys(['식료품', '비주류 음료', '주류', '담배', '의류', '신발', '주택임차료', '주거시설 유지 · 보수', '수도 및 주거관련 서비스', '전기 · 가스 및 기타연료', '가구 · 가사비품 및 카페트', '가정용 섬유제품', '가정용 기기', '주방용품 및 가정용품', '가정 · 정원용 공구 및 장비'])